Nowadays, safe driving has become a serious problem all over the country and even the world. Most of the consequences of traffic accidents are human factors, such as tired driving, talking on the phone while driving, talking back and so on. In order to reduce traffic accidents caused by human factors, a driver behavior detection method based on convolutional neural networks (CNN) and data enhancement is proposed. Based on the original convolutional neural network, this method optimized the original convolutional neural network and designed a model suitable for the task. At the same time, the data enhancement transformation method is introduced to expand the original data to improve the over-fitting problem caused by the lack of data. Experimental results show that the accuracy of the proposed model is improved by about 0.67% compared with the original model, and the results of the enhanced data are improved by 0.8% compared with the original data.

1.Import packages ,read file Information and display labels

In [ ]:
# -*- coding: utf-8 -*-
import os
from glob import glob
import random
import time
import tensorflow
import datetime
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # 3 = INFO, WARNING, and ERROR messages are not printed

from tqdm import tqdm

import numpy as np
import pandas as pd
from IPython.display import FileLink
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns 
from IPython.display import display, Image
import matplotlib.image as mpimg
import cv2

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_files       
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.metrics import log_loss

from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications.vgg16 import VGG16


dataset = pd.read_csv('../input/state-farm-distracted-driver-detection/driver_imgs_list.csv')
by_drivers = dataset.groupby('subject')
unique_drivers = by_drivers.groups.keys()
print(unique_drivers)
# Load the dataset previously downloaded from Kaggle
NUMBER_CLASSES = 10
# Color type: 1 - grey, 3 - rgb

2.Load data

Loads training and test datasets and displays dataset information

Dataset:State Farm, an insurance company, published a collection of 40,000 images of real-life drivers' behaviour.

In [ ]:
def get_cv2_image(path, img_rows, img_cols, color_type=3):
    # Loading as Grayscale image
    if color_type == 1:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    elif color_type == 3:
        img = cv2.imread(path, cv2.IMREAD_COLOR)
    # Reduce size
    img = cv2.resize(img, (img_rows, img_cols)) 
    return img

# Load training data
def load_train(img_rows, img_cols, color_type=3):
    start_time = time.time()
    train_images = [] 
    train_labels = []
    # Loop over the training folder 
    for classed in tqdm(range(NUMBER_CLASSES)):
        print('Loading directory c{}'.format(classed))
        file_path = os.path.join('..', 'input', 'state-farm-distracted-driver-detection', 'imgs','train', 'c' + str(classed), '*.jpg')
        #print('file_path:',file_path)
        files = glob(file_path)
        print('files len:',len(files))
        #files = glob(os.path.join('.', 'imgs','train', 'c' + str(classed), '*.jpg'))
        for file in files:
            img = get_cv2_image(file, img_rows, img_cols, color_type)
            #print("img:",img)
            #print("classed:",classed)
            train_images.append(img)
            train_labels.append(classed)
    print("Data Loaded in {} second".format(time.time() - start_time))
    return train_images, train_labels 
def read_and_normalize_train_data(img_rows, img_cols, color_type):
    X, labels = load_train(img_rows, img_cols, color_type)
    #print("labels:",labels);
    y = np_utils.to_categorical(labels, 10)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    x_train = np.array(x_train, dtype=np.uint8).reshape(-1,img_rows,img_cols,color_type)
    x_test = np.array(x_test, dtype=np.uint8).reshape(-1,img_rows,img_cols,color_type)
    
    return x_train, x_test, y_train, y_test

# Validation
def load_test(size=200000, img_rows=64, img_cols=64, color_type=3):
    path = os.path.join('..', 'input', 'state-farm-distracted-driver-detection', 'imgs', 'test', '*.jpg')
    files = sorted(glob(path))
    X_test, X_test_id = [], []
    total = 0
    files_size = len(files)
    for file in tqdm(files):
        if total >= size or total >= files_size:
            break
        file_base = os.path.basename(file)
        img = get_cv2_image(file, img_rows, img_cols, color_type)
        X_test.append(img)
        X_test_id.append(file_base)
        total += 1
    return X_test, X_test_id

def read_and_normalize_sampled_test_data(size, img_rows, img_cols, color_type=3):
    test_data, test_ids = load_test(size, img_rows, img_cols, color_type)
    
    test_data = np.array(test_data, dtype=np.uint8)
    test_data = test_data.reshape(-1,img_rows,img_cols,color_type)
    
    return test_data, test_ids


img_rows = 64
img_cols = 64
color_type = 1

x_train, x_test, y_train, y_test = read_and_normalize_train_data(img_rows, img_cols, color_type)
print('Train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')

nb_test_samples = 200
test_files, test_targets = read_and_normalize_sampled_test_data(nb_test_samples, img_rows, img_cols, color_type)
print('Test shape:', test_files.shape)
print(test_files.shape[0], 'Test samples')

# Statistics
# Load the list of names
names = [item[17:19] for item in sorted(glob("../input/state-farm-distracted-driver-detection/imgs/train/*/"))]
test_files_size = len(np.array(glob(os.path.join('..','input','state-farm-distracted-driver-detection','imgs', 'test', '*.jpg'))))
x_train_size = len(x_train)
categories_size = len(names)
x_test_size = len(x_test)
print('There are %s total images.\n' % (test_files_size + x_train_size + x_test_size))
print('There are %d training images.' % x_train_size)
print('There are %d total training categories.' % categories_size)
print('There are %d validation images.' % x_test_size)
print('There are %d test images.'% test_files_size)


3.Data visualization

The data set is divided into 10 categories:

c0: safe driving,c1: texting - right,c2: talking on the phone - right,c3: texting - left,c4: talking on the phone - left,c5: operating the radio,c6: drinking,c7: reaching behind,c8: hair and makeup,c9: talking to passenger


One of these 10 classes is randomly selected to show the data set.


In [ ]:
# Plot figure size
plt.figure(figsize = (10,10))
# Count the number of images per category
sns.countplot(x = 'classname', data = dataset)
# Change the Axis names
plt.ylabel('Count')
plt.title('Categories Distribution')
# Show plot
plt.show()

# Find the frequency of images per driver找出每个驱动程序的图像频率
drivers_id = pd.DataFrame((dataset['subject'].value_counts()).reset_index())
drivers_id.columns = ['driver_id', 'Counts']


# Plotting class distribution画出每一类的分布
dataset['class_type'] = dataset['classname'].str.extract('(\d)',expand=False).astype(np.float)
plt.figure(figsize = (20,20))
dataset.hist('class_type', alpha=0.5, layout=(1,1), bins=10)
plt.title('Class distribution')
plt.show()


activity_map = {'c0': 'Safe driving', 
                'c1': 'Texting - right', 
                'c2': 'Talking on the phone - right', 
                'c3': 'Texting - left', 
                'c4': 'Talking on the phone - left', 
                'c5': 'Operating the radio', 
                'c6': 'Drinking', 
                'c7': 'Reaching behind', 
                'c8': 'Hair and makeup', 
                'c9': 'Talking to passenger'}



plt.figure(figsize = (12, 20))
image_count = 1
BASE_URL = '../input/state-farm-distracted-driver-detection/imgs/train/'
for directory in os.listdir(BASE_URL):
    if directory[0] != '.':
        for i, file in enumerate(os.listdir(BASE_URL + directory)):
            if i == 1:
                break
            else:
                fig = plt.subplot(5, 2, image_count)
                image_count += 1
                image = mpimg.imread(BASE_URL + directory + '/' + file)
                plt.imshow(image)
                plt.title(activity_map[directory])
                
                
                

def create_submission(predictions, test_id, info):
    result = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
    result.loc[:, 'img'] = pd.Series(test_id, index=result.index)
    
    now = datetime.datetime.now()
    
    if not os.path.isdir('kaggle_submissions'):
        os.mkdir('kaggle_submissions')

    suffix = "{}_{}".format(info,str(now.strftime("%Y-%m-%d-%H-%M")))
    sub_file = os.path.join('kaggle_submissions', 'submission_' + suffix + '.csv')
    
    result.to_csv(sub_file, index=False)
    
    return sub_file


4.Model building

The original neural network model, denoted as Model1. BN layer is introduced into the optimized CNN network, denoted as Model2. On the basis of the optimized CNN model, the data input to the model is denoted as Model3 by using the data enhancement method to avoid over-fitting and merging to increase the generalization ability of the model.

Model3 is compared with the original convolutional neural network model (Model1) and the optimized convolutional neural network model (Model2) for experimental analysis. All models use the method of 80% training data set and 20% validation data set to segment the training set. Experiments verify the accuracy and loss value of the training set to analyze the experimental results.

In [ ]:
batch_size = 40
nb_epoch = 10

if os.path.exists('./saved_models/weights_best_vanilla.hdf5'):
    os.remove('./saved_models/weights_best_vanilla.hdf5')
    
    

models_dir = "saved_models"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    
checkpointer = ModelCheckpoint(filepath='./saved_models/weights_best_vanilla.hdf5', 
                               monitor='val_loss', mode='min',
                               verbose=1, save_best_only=True)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
callbacks = [checkpointer, es]


def plot_train_history(history):
    # Summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # Summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    
def create_model_v1():
    # Vanilla CNN model
    #构建一个最简单的CNN 模型
    model = Sequential()

    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same', activation = 'relu', input_shape=(img_rows, img_cols, color_type)))
    model.add(MaxPooling2D(pool_size = 2))

    model.add(Conv2D(filters = 128, padding='same', kernel_size = 3, activation = 'relu'))
    model.add(MaxPooling2D(pool_size = 2))

    model.add(Conv2D(filters = 256, padding='same', kernel_size = 3, activation = 'relu'))
    model.add(MaxPooling2D(pool_size = 2))

    model.add(Conv2D(filters = 512, padding='same', kernel_size = 3, activation = 'relu'))
    model.add(MaxPooling2D(pool_size = 2))

    model.add(Dropout(0.5))#50%的神经元不工作,拿掉部分神经元防止过拟合

    model.add(Flatten())#卷积层到全连接层的过渡

    model.add(Dense(500, activation = 'relu'))#全连接层，500个节点，使用relu激活函数
    model.add(Dropout(0.5))
    model.add(Dense(10, activation = 'softmax'))
    
    return model


model_v1 = create_model_v1()

# More details about the layers
model_v1.summary()

# Compiling the model
#定义优化器，损失函数，训练效果中计算准确率
#使用'rmsprop'优化器，loss = 损失用交叉熵，速度会更快，metrics计算准确率
model_v1.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


# Training the Vanilla Model version 1
#训练
history_v1 = model_v1.fit(x_train, y_train, 
          validation_data=(x_test, y_test),
          callbacks=[checkpointer],
          epochs=10, batch_size=batch_size, verbose=1)


model_v1.load_weights('./saved_models/weights_best_vanilla.hdf5')



def plot_test_class(model, test_files, image_number, color_type=1):
    print("test_files len:",len(test_files))
    img_brute = test_files[image_number]
    img_brute = cv2.resize(img_brute,(img_rows,img_cols))
    plt.imshow(img_brute, cmap='gray')

    new_img = img_brute.reshape(-1,img_rows,img_cols,color_type)
    #predict预测所提供数据的推断中最后一层的输出
    y_prediction = model.predict(new_img, batch_size=batch_size, verbose=1)
    print('Y prediction: {}'.format(y_prediction))
    print('Predicted: {}'.format(activity_map.get('c{}'.format(np.argmax(y_prediction)))))
    
    plt.show()
    
score = model_v1.evaluate(x_test, y_test, verbose=1)
print('Score: ', score)

plot_train_history(history_v1)

plot_test_class(model_v1, test_files, 20)

predictions_v1 = model_v1.predict(test_files, batch_size=batch_size)
FileLink(create_submission(predictions_v1, test_targets, score[0]))


if os.path.exists('./saved_models/weights_best_vanilla.hdf5'):
    os.remove('./saved_models/weights_best_vanilla.hdf5')
    
    
    
def create_model_v2():
    # Optimised Vanilla CNN model 优化模型
    model = Sequential()

    ## CNN 1
    model.add(Conv2D(32,(3,3),activation='relu',input_shape=(img_rows, img_cols, color_type)))
    model.add(BatchNormalization())#标准化层
    model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization(axis = 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.3))

    ## CNN 2
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization(axis = 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.3))

    ## CNN 3
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization(axis = 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.5))

    ## Output
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(10,activation='softmax'))

    return model


model_v2 = create_model_v2()

# More details about the layers
model_v2.summary()

# Compiling the model
model_v2.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


# Training the Vanilla Model
history_v2 = model_v2.fit(x_train, y_train, 
          validation_data=(x_test, y_test),
          callbacks=[checkpointer],
          epochs=nb_epoch, batch_size=batch_size, verbose=1)


plot_train_history(history_v2)

#model_v2.load_weights('saved_models/weights_best_vanilla.hdf5')#读取权重文件
model_v2.load_weights('./saved_models/weights_best_vanilla.hdf5')#读取权重文件

score = model_v2.evaluate(x_test, y_test, verbose=1)#模型评估
print('Score: ', score)



#生成目标文件
predictions_v2 = model_v2.predict(test_files, batch_size=batch_size)
FileLink(create_submission(predictions_v2, test_targets, score[0]))


#预测
y_pred = model_v2.predict(x_test, batch_size=batch_size, verbose=1)
score = log_loss(y_test, y_pred)
print('Score log loss:', score)


plot_test_class(model_v2, test_files, 101) # The model really performs badly这个型号的性能真的很差
plot_test_class(model_v2, test_files, 1) # The model really performs badly这个模型的性能真的很差
plot_test_class(model_v2, test_files, 143) 


# if os.path.exists('./saved_models/weights_best_vanilla.hdf5'):
#     os.remove('./saved_models/weights_best_vanilla.hdf5')
#     print('文件已删除')
    
    
# Prepare data augmentation configuration
train_datagen = ImageDataGenerator(rescale = 1.0/255, 
                                   #zoom_range = 0.25, 
                                   validation_split = 0.2)

test_datagen = ImageDataGenerator(rescale=1.0/ 255,
                                  validation_split = 0.2)

nb_train_samples = x_train.shape[0]
nb_validation_samples = x_test.shape[0]

#print(nb_train_samples)
#print(nb_validation_samples)

training_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
validation_generator = test_datagen.flow(x_test, y_test, batch_size=batch_size)


checkpoint = ModelCheckpoint('./saved_models/weights_best_vanilla.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
history_v3 = model_v2.fit_generator(training_generator,
                         steps_per_epoch = nb_train_samples // batch_size,
                         epochs = 10, 
                         callbacks= [checkpoint],
                         verbose = 1,
                         validation_data = validation_generator,
                         validation_steps = nb_validation_samples // batch_size)


#model_v2.load_weights('saved_models/weights_best_vanilla.hdf5')
model_v2.load_weights('./saved_models/weights_best_vanilla.hdf5')

plot_train_history(history_v3)
# Evaluate the performance of the new model评估新模型的性能
score = model_v2.evaluate_generator(validation_generator, nb_validation_samples // batch_size)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

plot_test_class(model_v2, test_files, 101)
plot_test_class(model_v2, test_files, 1)
plot_test_class(model_v2, test_files, 145)
plot_test_class(model_v2, test_files, 143) 

predictions_v3 = model_v2.predict(test_files, batch_size=batch_size)
FileLink(create_submission(predictions_v3, test_targets, score[0]))



5. Experimental comparison of Model1, Model2 and Model3.

In [ ]:
def plot_train_history(history_v1,history_v2,history_v3):
    # Summarize history for accuracy
    plt.plot(history_v1.history['val_accuracy'])
    plt.plot(history_v2.history['val_accuracy'])
    plt.plot(history_v3.history['val_accuracy'])
    #plt.title('Model accuracy')
    plt.ylabel('val_accuracy')
    plt.xlabel('epoch')
    plt.legend(['Model1', 'Model2','Model3'], loc='upper left')
    plt.show()

    # Summarize history for loss
    plt.plot(history_v1.history['val_loss'])
    plt.plot(history_v2.history['val_loss'])
    plt.plot(history_v3.history['val_loss'])
    #plt.title('Model loss')
    plt.ylabel('val_loss')
    plt.xlabel('epoch')
    plt.legend(['Model1', 'Model2','Model3'], loc='upper left')
    plt.show()
    
    
    
plot_train_history(history_v1,history_v2,history_v3)
